In [11]:
import sys
sys.path.append("../")

from PIL import Image

import torch
import torchvision
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset

from encoders import resnet18, resnet50
from aug import get_relic_aug_inference

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
IMAGE_SIZE = 32

transform = get_relic_aug_inference()
train_ds = torchvision.datasets.CIFAR10("../data/cifar",
                                  train=True,
                                  transform=transform,
                                  download=True)
val_ds = torchvision.datasets.CIFAR10("../data/cifar",
                                  train=False,
                                  transform=transform,
                                  download=True)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
len(train_ds), len(val_ds)

(50000, 10000)

In [15]:
# ckpt = torch.load("../models/sota_cifar_2048/encoder.pth")
ckpt = torch.load("../models/sota_cifar/encoder.pth")

# model = resnet50()
model = resnet18()
model.load_state_dict(ckpt)

model = model.eval().to(device)

In [16]:
from tqdm.auto import tqdm

def get_embs_targets(ds):
    idx = 0
    embs, targets = [], []
    for idx, (image, target) in enumerate(tqdm(ds)):
        with torch.no_grad():
            image = image.unsqueeze(0).to(device)
            out = model(image)
            features = out[0].cpu().detach().tolist()   
            embs.append(features)
            targets.append(target)
    return embs, targets

In [17]:
embs, targets = get_embs_targets(train_ds)
embs_val, targets_val = get_embs_targets(val_ds)

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [18]:
import numpy as np

embeddings = np.array(embs)
labels = np.array(targets)
embeddings_val = np.array(embs_val)
labels_val = np.array(targets_val)

In [19]:
print(embeddings.shape)
print(labels.shape)
print(embeddings_val.shape)
print(labels_val.shape)

(50000, 512)
(50000,)
(10000, 512)
(10000,)


# ResNet-18

In [10]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
 
X_train, X_test = embeddings, embeddings_val
y_train, y_test = labels, labels_val

print("train", X_train.shape[0], len(y_train))
print("test", X_test.shape[0], len(y_test))
 
clf = LogisticRegression(max_iter=100)
clf = CalibratedClassifierCV(clf)
 
clf.fit(X_train, y_train)
 
y_pred = clf.predict(X_test)
 
acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
 
print("Accuracy: ", acc)
print("Confusion matrix: \n", conf_matrix)
print("Classification report: \n", class_report)
 
y_pred_train = clf.predict(X_train)
class_report = classification_report(y_train, y_pred_train)
print("Classification report train: \n", class_report)

train 50000 50000
test 10000 10000


/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Accuracy:  0.697
Confusion matrix: 
 [[718  14  70  16  21   6  15  14  94  32]
 [ 24 813   9  10   7   6  13   3  35  80]
 [ 62  12 566  75  86  63  84  31  11  10]
 [ 15   8  73 498  63 155 106  56   8  18]
 [ 20   4  66  56 619  41  80  96  15   3]
 [  8   7  56 183  37 565  60  68   7   9]
 [ 10   4  35  54  30  32 818  11   4   2]
 [ 14   6  35  44  58  60  19 748   4  12]
 [ 59  36  13  21   8   2   7   2 833  19]
 [ 24  82  14  19   5   8   9  20  27 792]]
Classification report: 
               precision    recall  f1-score   support

           0       0.75      0.72      0.73      1000
           1       0.82      0.81      0.82      1000
           2       0.60      0.57      0.58      1000
           3       0.51      0.50      0.50      1000
           4       0.66      0.62      0.64      1000
           5       0.60      0.56      0.58      1000
           6       0.68      0.82      0.74      1000
           7       0.71      0.75      0.73      1000
           8       0